In [ ]:
!pip install pyvi
import os
import csv
from pyvi import ViTokenizer
import random
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 26.2 MB/s 
     |████████████████████████████████| 965 kB 45.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Convert multiple lines raw text to single line

In [ ]:
DATA_TYPE = '/content/drive/MyDrive/vncorenlp/journal_hero/punctuation/Novels/generated'
url_pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
email_pattern = r"^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w+$"
for name in ['qmark', 'colon', 'semicolon', 'exclam']:
    out_file = open(f'{DATA_TYPE}/train_{name}_line.txt', 'w', encoding='utf-8')
    with open(f'{DATA_TYPE}/new_{name}_data.txt', 'r', encoding='utf-8') as file:
        cnt = 0
        for line in file:
            line = " ".join(line.strip().split()).replace('…', '.')
            line = re.sub(r'([.,-?!:;\'~_+]){2,}', r'\1', line)
            line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
            tokened_lines = ViTokenizer.tokenize(line.strip().lower()).replace('_', ' ')
            tokens = tokened_lines.split()
            for t in tokens:
                if t == '\u200e': continue
                if re.search(email_pattern, t) is not None:
                    t = '<email_obj>'
                elif re.search(url_pattern, t) is not None:
                    t = '<url_obj>'

                if re.search(r'(\d[.,]?)+', t) is not None: # is number
                    if t.count('.') > 1 or t.count('.') > t.count(',') >= 1:
                        t = " , ".join(t.replace('.', '').split(','))

                    elif t.count(',') > 1 or 1 <= t.count('.') < t.count(','):
                        t = " , ".join(t.replace(',', '').split('.'))
                    else:
                        if re.search(r'(\d+\,\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace('.', '').split(','))
                        elif re.search(r'(\d+\.\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace(',', '').split('.'))
                        else:
                            t = t.replace('.', '').replace(',','')

                out_file.write(t + ' ')
    out_file.close()

In [ ]:
punctuation_labels = {
    ',': 'COMMA',
    '.': 'PERIOD',
    ':': 'COLON',
    '?': 'QMARK',
    '!': 'EXCLAM',
    ';': 'SEMICOLON'
}
DATA_TYPE = '/content/drive/MyDrive/vncorenlp/journal_hero/punctuation/Novels/generated'
for prefix in ['qmark', 'colon', 'semicolon', 'exclam']:
    with open(f'{DATA_TYPE}/train_{prefix}.txt', 'w', encoding='utf-8') as out_file:
        with open(f'{DATA_TYPE}/train_{prefix}_line.txt', 'r', encoding='utf-8') as file:
            for line in file:
                while re.search(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', line):
                    line = ' '.join(line.split())
                    line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
                    
                tokens = line.strip().split()
                label = None
                is_prev_mark = False
                for idx, token in enumerate(tokens):
                    prev_label = label
                    label = punctuation_labels.get(token, 'O')
                    if label != 'O':
                        out_file.write(label + '\n')
                    else:
                        if prev_label is not None and prev_label == 'O':
                            out_file.write(label + '\n')
                        out_file.write(token + ' ')


In [ ]:
!head -n 10 $DATA_TYPE/new_exclam_data.txt

– tiểu long cười hích hích – bọn tao có bảo mày lén uống lọ thuốc để trên đầu tủ đâu!
đô mi nô thuyết phục dũng cò đồng ý cho tụi mình chuộc lại thôi!
Anh chị này lại đi theo con mắt thứ ba nữa!
ừ, nhưng lần này không phải là thơ lục bát nữa đâu!
Nó hay đánh bạn, phải cho nó biết, nếu không nó sẽ làm nhiều tai nạn nữa!
chiếc xe của em buộc mấy chục quả bongbóng còn không bay lên nổi nữa là!
đẹp lộng lẫy!
Vậy đó! Thế nên, anh nói vô, mình nói đáp án vậy thôi, có nghĩ ra đâu mà hỏi!
Nghe đây!
lần này là lần chót nghen, cô giáo!


In [ ]:
!head -n 100 $DATA_TYPE/train_exclam.txt

– O
tiểu O
long O
cười O
hích O
hích O
– O
bọn O
tao O
có O
bảo O
mày O
lén O
uống O
lọ O
thuốc O
để O
trên O
đầu O
tủ O
đâu EXCLAM
đô O
mi O
nô O
thuyết O
phục O
dũng O
cò O
đồng O
ý O
cho O
tụi O
mình O
chuộc O
lại O
thôi EXCLAM
anh O
chị O
này O
lại O
đi O
theo O
con O
mắt O
thứ O
ba O
nữa EXCLAM
ừ COMMA
nhưng O
lần O
này O
không O
phải O
là O
thơ O
lục O
bát O
nữa O
đâu EXCLAM
nó O
hay O
đánh O
bạn COMMA
phải O
cho O
nó O
biết COMMA
nếu O
không O
nó O
sẽ O
làm O
nhiều O
tai O
nạn O
nữa EXCLAM
chiếc O
xe O
của O
em O
buộc O
mấy O
chục O
quả O
bongbóng O
còn O
không O
bay O
lên O
nổi O
nữa O
là EXCLAM
đẹp O
lộng O
lẫy EXCLAM
vậy O
đó EXCLAM
thế O
nên COMMA
anh O


### ???

In [ ]:
no_dup = set()
with open('Subtitles/train_raw.txt', 'r', encoding='utf-8') as file:
    for line in file:
        no_dup.add(line.strip())

In [ ]:
out_file = open(f'Subtitles/tmp/test_raw.txt', 'w', encoding='utf-8')
for filename in os.listdir('Subtitles/tmp/'):
    if filename.endswith('srt'):
        with open('Subtitles/tmp/' + filename, 'r', encoding='utf-8') as file:
            cnt = 0
            for line in file:
                line = " ".join(line.strip().split()).replace('…', '.')
                if line.count(' ') < 3:
                    continue
                if '-->' in line:
                    continue
                line = line.replace('<i>', '').replace('</i>', '')
                if line not in no_dup:
                    out_file.write(line + '\n')
                    no_dup.add(line)
out_file.close()

In [ ]:
for filename in os.listdir('Subtitles/tmp/'):
    out_file = open(f'Subtitles/tmp/test_raw.txt', 'w', encoding='utf-8')
    with open('Subtitles/tmp/' + filename, 'r', encoding='utf-8') as file:
        cnt = 0
        for line in file:
            line = " ".join(line.strip().split()).replace('…', '.')
            if line.count(' ') < 3:
                continue
            if '-->' in line:
                continue
            line = re.sub(r'([.,-?!:;\'~_+]){2,}', r'\1', line)
            line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
            tokened_lines = ViTokenizer.tokenize(line.strip().lower()).replace('_', ' ')
            tokens = tokened_lines.split()
            for t in tokens:
                if re.search(email_pattern, t) is not None:
                    t = '<email_obj>'
                elif re.search(url_pattern, t) is not None:
                    t = '<url_obj>'

                if re.search(r'(\d[.,]?)+', t) is not None: # is number
                    if t.count('.') > 1 or t.count('.') > t.count(',') >= 1:
                        t = " , ".join(t.replace('.', '').split(','))

                    elif t.count(',') > 1 or 1 <= t.count('.') < t.count(','):
                        t = " , ".join(t.replace(',', '').split('.'))
                    else:
                        if re.search(r'(\d+\,\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace('.', '').split(','))
                        elif re.search(r'(\d+\.\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace(',', '').split('.'))
                        else:
                            t = t.replace('.', '').replace(',','')

                out_file.write(t + ' ')

    out_file.close()

In [ ]:
random.seed(0)
filenames = os.listdir('text/')
filenames = random.sample(filenames, int(len(filenames)*0.65))
len(filenames), filenames[:5]

In [ ]:
train_filenames = random.sample(filenames, int(len(filenames)*0.8))
tmp = set(filenames)-set(train_filenames)
test_filenames = random.sample(list(tmp), k=int(len(tmp)*0.5))
valid_filenames = set(tmp)-set(test_filenames)
len(test_filenames), len(train_filenames), len(valid_filenames)

### Convert to sequence tagging

In [ ]:
dir_path = 'text/'
DATA_TYPE = 'News'
url_pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
email_pattern = r"^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w+$"
for name, s in zip(['train', 'test', 'valid'], [train_filenames, test_filenames, valid_filenames]):
    out_file = open(f'{DATA_TYPE}/{name}_raw.txt', 'w', encoding='utf-8')
    for filename in s:
        with open(dir_path + filename, 'r', encoding='utf-8') as file:
            cnt = 0
            skip_header = True
            for line in file:
                if skip_header:
                    skip_header = False
                    continue

                line = " ".join(line.strip().split()).replace('…', '.')
                line = re.sub(r'([.,-?!:;\'~_+]){2,}', r'\1', line)
                line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
                tokened_lines = ViTokenizer.tokenize(line.strip().lower()).replace('_', ' ')
                tokens = tokened_lines.split()
                for t in tokens:
                    if re.search(email_pattern, t) is not None:
                        t = '<email_obj>'
                    elif re.search(url_pattern, t) is not None:
                        t = '<url_obj>'

                    if re.search(r'(\d[.,]?)+', t) is not None: # is number
                        if t.count('.') > 1 or t.count('.') > t.count(',') >= 1:
                            t = " , ".join(t.replace('.', '').split(','))

                        elif t.count(',') > 1 or 1 <= t.count('.') < t.count(','):
                            t = " , ".join(t.replace(',', '').split('.'))
                        else:
                            if re.search(r'(\d+\,\d{1,2})$', t) is not None:
                                t = " , ".join(t.replace('.', '').split(','))
                            elif re.search(r'(\d+\.\d{1,2})$', t) is not None:
                                t = " , ".join(t.replace(',', '').split('.'))
                            else:
                                t = t.replace('.', '').replace(',','')

                    out_file.write(t + ' ')

    out_file.close()

In [ ]:
DATA_TYPE = 'Subtitles'
url_pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
email_pattern = r"^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w+$"
for name in ['train', 'test']:
    out_file = open(f'{DATA_TYPE}/{name}_all.txt', 'w', encoding='utf-8')
    with open(f'{DATA_TYPE}/{name}_raw.txt', 'r', encoding='utf-8') as file:
        cnt = 0
        for line in file:
            line = " ".join(line.strip().split()).replace('…', '.')
            line = re.sub(r'([.,-?!:;\'~_+]){2,}', r'\1', line)
            line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
            tokened_lines = ViTokenizer.tokenize(line.strip().lower()).replace('_', ' ')
            tokens = tokened_lines.split()
            for t in tokens:
                if t == '\u200e': continue
                if re.search(email_pattern, t) is not None:
                    t = '<email_obj>'
                elif re.search(url_pattern, t) is not None:
                    t = '<url_obj>'

                if re.search(r'(\d[.,]?)+', t) is not None: # is number
                    if t.count('.') > 1 or t.count('.') > t.count(',') >= 1:
                        t = " , ".join(t.replace('.', '').split(','))

                    elif t.count(',') > 1 or 1 <= t.count('.') < t.count(','):
                        t = " , ".join(t.replace(',', '').split('.'))
                    else:
                        if re.search(r'(\d+\,\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace('.', '').split(','))
                        elif re.search(r'(\d+\.\d{1,2})$', t) is not None:
                            t = " , ".join(t.replace(',', '').split('.'))
                        else:
                            t = t.replace('.', '').replace(',','')

                out_file.write(t + ' ')
    out_file.close()

In [ ]:
punctuation_labels = {
    ',': 'COMMA',
    '.': 'PERIOD',
    ':': 'COLON',
    '?': 'QMARK',
    '!': 'EXCLAM',
    ';': 'SEMICOLON'
}
DATA_TYPE = 'Subtitles'
for prefix in ['test', 'train']:
    with open(f'{DATA_TYPE}/{prefix}.txt', 'w', encoding='utf-8') as out_file:
        with open(f'{DATA_TYPE}/{prefix}_all.txt', 'r', encoding='utf-8') as file:
            for line in file:
                while re.search(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', line):
                    line = ' '.join(line.split())
                    line = re.sub(r'([.,-?!:;\'~_+]) ([.,-?!:;\'~_+])', r'\1', line)
                    
                tokens = line.strip().split()
                label = None
                is_prev_mark = False
                for idx, token in enumerate(tokens):
                    prev_label = label
                    label = punctuation_labels.get(token, 'O')
                    if label != 'O':
                        out_file.write(label + '\n')
                    else:
                        if prev_label is not None and prev_label == 'O':
                            out_file.write(label + '\n')
                        out_file.write(token + ' ')


In [ ]:
with open('Subtitles/test.txt', 'r', encoding='utf-8') as file:
    cnt = 0
    tokens, labels = [], []
    for idx, line in enumerate(file):
        if ' ' not in line:
            print(idx)
            cnt += 1
cnt


0

In [ ]:
from collections import Counter

In [ ]:
l = []
with open('Subtitles/test.txt', 'r', encoding='utf-8') as f:
    cnt = 0
    for line in f:
        cnt += 1
        if ' ' not in line:
            print(cnt)
        a, b = line.strip().split()
        l.append(b)
        
Counter(l)

Counter({'O': 71789,
         'QMARK': 2145,
         'COMMA': 2652,
         'PERIOD': 8154,
         'EXCLAM': 671,
         'COLON': 44})

In [ ]:




end_sentence_marks = ['.', '!', '?']

for filename in os.listdir(dir_path):
    with open(dir_path + filename, 'r', encoding='utf-8') as file:
        cnt = 0
        skip_header = True
        for line in file:
            if skip_header:
                skip_header = False
                continue
            
            line = " ".join(line.strip().split())
            line = re.sub(r'([.,]){2,}', r'\1', line).replace('…', '.')
            
            tokened_lines = ViTokenizer.tokenize(line.strip().lower()).replace('_', ' ')
            tokens = tokened_lines.split()
            tokens.append('<ENDLINE>')
            label = None
            is_prev_mark = True
            for idx, token in enumerate(tokens):
                label = punctuation_labels.get(token, 'O')
                is_write = False
                if not is_prev_mark:
                    if label == '<ENDLINE>':
                        out_file.write('O' + '\n')
                    else:
                        out_file.write(label + '\n')
                    is_write = True
                if label != 'O':
                    is_prev_mark = True

                else:
                    is_prev_mark = False
                    out_file.write(token + ' ')
    
out_file.close()

### Split comma, period number seperate

In [ ]:
punctuation_labels = {
    ',': 'COMMA',
    '.': 'PERIOD',
    ':': 'COLON',
    '?': 'QMARK',
    '!': 'EXCLAM',
    ';': 'SEMICOLON',
    '<ENDLINE>': '<ENDLINE>'
}
out_file = open('cleaned_BM_split.txt', 'w', encoding='utf-8')
with open('cleaned_BM.txt', 'r', encoding='utf-8') as file:
    cnt = 0
    for line in file:
        cnt += 1
        line = line.strip()
        
        if re.search(r'[\.\?\!,:;]', line) is not None:
            token, label = line.split()
            sub_token = ''
            for c in token:
                if c not in punctuation_labels:
                    sub_token += c
                else:
                    out_file.write(sub_token + ' ' + punctuation_labels[c] + '\n')
                    sub_token = ''
            if sub_token != '':
                out_file.write(sub_token + ' ' + label + '\n')
        else:
            out_file.write(line + '\n')
            
out_file.close()

### Convert unique tokens (number, email, url) into special tokens

In [ ]:
NUM = '<NUM>'
EMAIL = '<EMAIL>'
URL = '<URL>'
out_file = open('cleaned_BM_special.txt', 'w', encoding='utf-8')
with open('cleaned_BM.txt', 'r', encoding='utf-8') as file:
    cnt = 0
    for line in file:
        cnt += 1
        line = line.strip()
        token, label = line.split()
        if re.search(r'[^@]+@[^@]+\.[^@]+', token):
            
        
        if re.search(r'[\.\?\!,:;]', line) is not None:
            token, label = line.split()
            
            
            out_file.write(new_token + ' ' + label + '\n')
        else:
            out_file.write(line + '\n')
            
out_file.close()

In [ ]:
with open('clean_news.txt', 'r', encoding='utf-8') as file:
    cnt = 0
    n_label = 0
    for line in file:
        _, l = line.strip().split()
        if l != 'O':
            n_label += 1
#         print(line)
        cnt += 1

In [ ]:
cnt, n_label

In [ ]:
#News: 52tr tokens, 4tr labels
#Novels: 3tr tokens, 350k labels